In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## data procecessing

In [2]:
batch_size=8
width = 256
hight = 256
train_path = "D:/Projects/crowdai_train/train"
val_path = "D:/Projects/crowdai_train/val"

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=40,
        fill_mode='nearest',
        horizontal_flip=True,
        vertical_flip=True)
val_datagen = ImageDataGenerator(        
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=40,
        fill_mode='nearest',
        horizontal_flip=True,
        vertical_flip=True)

In [4]:
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(width,hight),
        batch_size=batch_size,
        class_mode='categorical')

Found 19739 images belonging to 38 classes.


In [5]:
val_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=(width,hight),
        batch_size=batch_size,
        class_mode='categorical')

Found 2178 images belonging to 38 classes.


## Resnet Model

In [6]:
from tensorflow.keras.applications.densenet import DenseNet121

In [7]:
densenet = DenseNet121(include_top=False,input_shape=(width,hight,3))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
densenet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 262, 262, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 128, 128, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalizationV1) (None, 128, 128, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(densenet)
model.add(Dense(32, activation='relu'))
model.add(Flatten())
model.add(Dense(38, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

In [ ]:
model.fit_generator(
        generator=train_generator,
        steps_per_epoch=1024//batch_size,
        epochs=50,
        validation_data=val_generator,
        validation_steps=128//batch_size
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
2468/2468 [==============================] - 596s 241ms/step - loss: 1.7207 - categorical_accuracy: 0.5052 - val_loss: 1.3987 - val_categorical_accuracy: 0.5468
Epoch 2/50
2468/2468 [==============================] - 493s 200ms/step - loss: 0.7891 - categorical_accuracy: 0.7522 - val_loss: 0.7458 - val_categorical_accuracy: 0.7769
Epoch 3/50
2468/2468 [==============================] - 492s 200ms/step - loss: 0.5542 - categorical_accuracy: 0.8219 - val_loss: 0.3822 - val_categorical_accuracy: 0.8747
Epoch 4/50
2468/2468 [==============================] - 491s 199ms/step - loss: 0.4356 - categorical_accuracy: 0.8625 - val_loss: 0.4320 - val_categorical_accuracy: 0.8600
Epoch 5/50
2468/2468 [==============================] - 492s 200ms/step - loss: 0.3712 - categorical_accuracy: 0.8807 - val_loss: 0.3823 - val_categorical_accuracy: 0.8852
Epoch 6/50
2468/2468 [==============================] - 492s 199ms/step - loss: 0.3079 - cat

In [ ]:
model.save("densenet.h5")

In [ ]:
history = model.history

In [ ]:
import pandas as pd
import sys,os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
history_df.to_csv("densenet.csv")

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
prediction = model.predict_generator(val_generator,steps=val_generator.samples/batch_size,verbose=0)

In [ ]:
y_pred = np.argmax(prediction,axis=1)
print('Confusion Matrix')
cnf_matrix = confusion_matrix(val_generator.classes, y_pred)
print(cnf_matrix)

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.hsv_r):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')


    plt.imshow(cm, cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
plt.rcParams["figure.figsize"] = (10,8)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=list(val_generator.class_indices.keys()),
                      title='Confusion matrix')